# Introduction

Welcome to the P1 Workspace with Group 14:
Will Mitchell
Trevor Gordon


Below are some prelimnary Helper methods that benifit the program as a whole with the starting code parameters

In [321]:
# Parameters
GROUP_ID = 'Group14'
ALGORITHM = 'bt'
PUZZLE_TYPE = 'easy'
PUZZLE_PATH = 'puzzles/Med-P1.txt'

In [322]:
def parse_matrix(file_path : str):
  with open(file_path, mode="r") as file:
      new_matrix = []
      for line in file:
        new_line_array = line.replace("\n","").replace("\ufeff","").split(",")
        new_matrix.append(new_line_array)


  file.close()
  new_matrix[0][0] = str(new_matrix[0][0])[-1:]
  return new_matrix

parse_matrix(PUZZLE_PATH)



[['2', '5', '?', '9', '?', '?', '7', '?', '?'],
 ['9', '?', '?', '1', '?', '?', '?', '?', '?'],
 ['?', '1', '?', '3', '?', '?', '?', '4', '?'],
 ['?', '4', '1', '7', '?', '8', '?', '?', '2'],
 ['?', '9', '?', '?', '?', '?', '?', '8', '?'],
 ['8', '?', '?', '5', '?', '2', '4', '3', '?'],
 ['?', '2', '?', '?', '?', '3', '?', '9', '?'],
 ['?', '?', '?', '?', '?', '9', '?', '?', '3'],
 ['?', '?', '9', '?', '?', '7', '?', '5', '4']]

In [323]:
def get_square_values(sudoku_state, row, col):
    #top left box
    if col <= 2 and row <= 2:
        col_indexes = [0,1,2]
        row_indexes = [0,1,2]

    #top middle box
    elif (2 < col <= 5) and (row <= 2):
        col_indexes = [3,4,5]
        row_indexes = [0,1,2]

    #top right box
    elif (5 < col <= 8) and (row <= 2):
        col_indexes = [6,7,8]
        row_indexes = [0,1,2]

    #middle left box
    elif (col <= 2) and (2 < row <= 5):
        col_indexes = [0,1,2]
        row_indexes = [3,4,5]

    #middle middle box
    elif (2 < col <= 5) and (2 < row <= 5):
        col_indexes = [3,4,5]
        row_indexes = [3,4,5]

    #middle right box
    elif (5 < col <= 8) and (2 < row <= 5):
        col_indexes = [6,7,8]
        row_indexes = [3,4,5]

    #bottom left box
    elif (col <= 2) and (row > 5):
        col_indexes = [0,1,2]
        row_indexes = [6,7,8]

    #bottom middle box
    elif (2 < col <= 5) and (row > 5):
        col_indexes = [3,4,5]
        row_indexes = [6,7,8]

    #bottom right box
    elif (5 < col <= 8) and (row > 5):
        col_indexes = [6,7,8]
        row_indexes = [6,7,8]

    my_set = set()
    for i in row_indexes:
            for j in col_indexes:
                if sudoku_state[i][j] != '?':
                      my_set.add(sudoku_state[i][j])
    return my_set



# print(get_square_values(parse_matrix(PUZZLE_PATH), 6, 1))



In [286]:
#TODO: THIS ONE IS BACKTRACKING I THINK

def is_available(sudoku_state, row, col, num):
    # uses get_unavailable_nums to return what nums are there already
    #then just returns true or false if num is available
    my_list = get_unavailable_nums(sudoku_state,row,col)
    if str(num) in my_list:
        return False
    else:
        return True


def solver_back(sudoku_state, row, col):
    #if sudoku board is complete
    if row == 8 and col == 9:
        return True

    # If last column of the row go to the next row
    if col == 9:
        row += 1
        col = 0

    # If spot is already filled then go to next
    if sudoku_state[row][col] != '?':
        return solver_back(sudoku_state, row, col+1)

    for num in range(1, 10):
        # If num is available in the current spot add it
        if is_available(sudoku_state, row, col, num):
            sudoku_state[row][col] = str(num)
            if solver_back(sudoku_state, row, col+1):
                return True
            sudoku_state[row][col] = '?'
    return False

def solveSudoku_back(sudoku_state): #starts recursive solving
    solver_back(sudoku_state, 0, 0)
    for line in sudoku_state:
        print(line)

solveSudoku_back(parse_matrix(PUZZLE_PATH))

['2', '5', '3', '9', '8', '4', '7', '1', '6']
['9', '8', '4', '1', '7', '6', '3', '2', '5']
['6', '1', '7', '3', '2', '5', '8', '4', '9']
['5', '4', '1', '7', '3', '8', '9', '6', '2']
['3', '9', '2', '6', '4', '1', '5', '8', '7']
['8', '7', '6', '5', '9', '2', '4', '3', '1']
['7', '2', '5', '4', '1', '3', '6', '9', '8']
['4', '6', '8', '2', '5', '9', '1', '7', '3']
['1', '3', '9', '8', '6', '7', '2', '5', '4']


In [327]:
#Todo: THIS ONE IS FORWARD CHECKING

def available(sudoku_state, row, col):
    unavail_list = get_unavailable_nums(sudoku_state,row,col)
    my_set = {'1', '2', '3', '4', '5', '6', '7', '8', '9'}
    my_set = my_set.difference(unavail_list)
    return my_set


def solver_for(sudoku_state, row, col):
    #if sudoku board is complete
    if row == 8 and col == 9:
        return True

    # If last column of the row go to the next row
    if col == 9:
        row += 1
        col = 0

    # If spot is already filled then go to next
    if sudoku_state[row][col] != '?':
        return solver_for(sudoku_state, row, col+1)

    #create domain of available numbers
    avail_list = available(sudoku_state, row, col)
    #test each available numbers recursively like normal back tracking
    for num in avail_list:
        #TODO: is this correct?
        sudoku_state[row][col] = str(num)
        #TODO ADD COUNTER HERE FOR EACH STEP
        if solver_for(sudoku_state, row, col+1):
            return True
        sudoku_state[row][col] = '?'
    return False

def solveSudoku_forward(sudoku_state): #starts recursive solving
    solver_for(sudoku_state, 0, 0)
    for line in sudoku_state:
        pass
        print(line)

solveSudoku_forward(parse_matrix(PUZZLE_PATH))

{'9', '5', '2', '?', '1', '7'}
{'3', '9', '5', '6', '2', '8', '1', '4', '7'}
{'6', '4', '8', '3'}

{'3', '9', '5', '6', '2', '?', '1', '7'}
{'3', '9', '5', '6', '2', '8', '1', '4', '7'}
{'4', '8'}

{'3', '9', '5', '6', '2', '?', '8', '1', '4', '7'}
{'3', '9', '5', '6', '2', '8', '1', '4', '7'}
set()

{'3', '9', '5', '6', '2', '?', '8', '1', '7'}
{'3', '9', '5', '6', '2', '8', '1', '4', '7'}
{'4'}

{'3', '9', '5', '6', '2', '?', '8', '4', '7'}
{'3', '9', '5', '6', '2', '8', '1', '4', '7'}
{'1'}

{'3', '9', '5', '6', '2', '?', '8', '1', '4', '7'}
{'3', '9', '5', '6', '2', '8', '1', '4', '7'}
set()

{'3', '9', '5', '2', '?', '1', '4', '7'}
{'3', '9', '5', '6', '2', '8', '1', '4', '7'}
{'6', '8'}

{'3', '9', '5', '6', '2', '?', '8', '1', '4', '7'}
{'3', '9', '5', '6', '2', '8', '1', '4', '7'}
set()

{'3', '9', '5', '2', '?', '8', '1', '4', '7'}
{'3', '9', '5', '6', '2', '8', '1', '4', '7'}
{'6'}

{'3', '9', '5', '6', '2', '?', '8', '4', '7'}
{'3', '9', '5', '6', '2', '8', '1', '4', '7'}
{'

In [328]:
#TODO: implement Arc Consistency!!!!!!!!!!!!!!!!!!!!!!

# local variables: queue, initially all edges in csp
# while queue is not empty do
# (X i, X j) := REMOVE-FIRST(queue)
# if REMOVE-INCONSISTENT-VALUES(X i, X j) then
# if size of DOMAIN[X i] = 0, then return false
# for each X k in NEIGHBORS[X i] do
# add (X k, X i) to queue
# return true
# function REMOVE-INCONSISTENT-VALUES(X i, X j)
# removed := false
# for each x in DOMAIN[X i] do
# if no value y in DOMAIN[X j] allows (x, y) to satisfy C(X i, X j)
# then delete x from DOMAIN[X i]; removed := true
# return removed
# 20
# Variable Assignment
# • Recall the line in simple backtracking:
# • var := SELECT-UNASSIGNED-VARIABLE(VARIABLES[csp],asg,csp)
# • Careful selection of which variables to assign can improve the performance of
# backtracking search.
# • Heuristics:
# • Minimum remaining values: choose the variable with the fewest “legal” values.
# • Degree heuristic: choose the variable involved in the largest number of constraints on
# other unassigned variables.
# • Least constraining value: prefer the value that rules out the fewest choices for
# neighboring unassigned variables.


def AvailableDomain(sudoku_state, row, col):
    unavail_list = get_unavailable_nums(sudoku_state,row,col)
    my_set = {'1', '2', '3', '4', '5', '6', '7', '8', '9'}
    my_set = my_set.difference(unavail_list)
    return my_set


def solver_back(sudoku_state, row, col):
    #if sudoku board is complete
    if row == 8 and col == 9:
        return True

    # If last column of the row go to the next row
    if col == 9:
        row += 1
        col = 0

    # If spot is already filled then go to next
    if sudoku_state[row][col] != '?':
        return solver_back(sudoku_state, row, col+1)

    #create domain of available numbers
    avail_list = AvailableDomain(sudoku_state, row, col)
    if len(avail_list) == 0:
        return False
    #test each available numbers recursively like normal back tracking
    for num in avail_list:
        #TODO: change
        sudoku_state[row][col] = str(num)
        if solver_back(sudoku_state, row, col+1):
            return True
        sudoku_state[row][col] = '?'
    return False

def solveSudoku_backtracking(sudoku_state): #starts recursive solving
    solver_back(sudoku_state, 0, 0)
    for line in sudoku_state:
        pass
        print(line)

solveSudoku_backtracking(parse_matrix(PUZZLE_PATH))

['2', '5', '3', '9', '8', '4', '7', '1', '6']
['9', '8', '4', '1', '7', '6', '3', '2', '5']
['6', '1', '7', '3', '2', '5', '8', '4', '9']
['5', '4', '1', '7', '3', '8', '9', '6', '2']
['3', '9', '2', '6', '4', '1', '5', '8', '7']
['8', '7', '6', '5', '9', '2', '4', '3', '1']
['7', '2', '5', '4', '1', '3', '6', '9', '8']
['4', '6', '8', '2', '5', '9', '1', '7', '3']
['1', '3', '9', '8', '6', '7', '2', '5', '4']


In [ ]:
#TODO: this is ac3 from almost scratch
def AvailableDomain(sudoku_state, row, col):
    unavail_list = get_unavailable_nums(sudoku_state,row,col)
    my_set = {'1', '2', '3', '4', '5', '6', '7', '8', '9'}
    my_set = my_set.difference(unavail_list)
    return my_set

def solver_back(sudoku_state, row, col):


def solveSudoku_backtracking(sudoku_state): #starts recursive solving
    solver_back(sudoku_state, 0, 0)
    for line in sudoku_state:
        pass
        print(line)

solveSudoku_backtracking(parse_matrix(PUZZLE_PATH))




In [165]:
class ga_state_node():
    def __init__(self, raw_matrix):
        self.matrix = raw_matrix
        self.num_conflicts = [-1] #this will track actually the history of conflicts, and push new conflict values each time its processed per generation
        self.hardcoded_value_map = self.process_hardcoded_values_map()
        #we want to immediately be able to know which indexes are hardcoded or not for a basic constraint

    def process_conflicts(self):
        self.num_conflicts.push(1)

    def process_hardcoded_values_map(self):
        return ((0,0), (0,1), (1,0), (1,1))
        #map of our indexes

    def fill_random_values(self):
        self.matrix = [] #fills question marks with random values


In [310]:
A = {10, 20, 30, 40, 80}
B = {100, 30, 80, 40, 60}

print(A.difference(B))  # Elements in A but not in B
print(B.difference(A))

{10, 20}
{100, 60}


# Helper Functions


In [324]:
def get_row_values(sudoku_state, row, col):
  my_set = set(sudoku_state[row])
  return my_set

# print(get_row_values(parse_matrix(PUZZLE_PATH),0,0))

In [325]:
def get_col_values(sudoku_state, row, col):
  my_set = set()
  for i in range(9):
      my_set.add(sudoku_state[i][col])


  return my_set

print(get_col_values(parse_matrix(PUZZLE_PATH),0,0))

{'2', '?', '9', '8'}


In [326]:
def get_unavailable_nums(sudoku_state, row, col):
    unavail_row = get_row_values(sudoku_state,row, col)
    unavail_col = get_col_values(sudoku_state,row, col)
    unavail_square = get_square_values(sudoku_state, row, col)
    my_set = unavail_row.union(unavail_col, unavail_square)
    return my_set

get_unavailable_nums(parse_matrix(PUZZLE_PATH),0,2)

{'1', '2', '5', '7', '9', '?'}